In [1]:
import torch
print(torch.backends.mps.is_available())

True


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

device = torch.device('mps')

# 2. 定义超参数
input_size = 784  # MNIST图片大小是28x28
hidden_sizes = [128, 64]  # 隐藏层的大小
output_size = 10  # 输出的类别数为10，分别对应0到9的数字
num_epochs = 50  # 进行5次训练迭代
batch_size = 64  # 每批次处理64张图片
learning_rate = 0.001  # 学习率设置为0.001

# 3. 数据准备
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换成PyTorch的Tensor格式
    transforms.Normalize((0.5,), (0.5,))  # 标准化处理，以减小模型对数据规模的敏感性
])

# 加载数据集，这里使用内置的MNIST数据集
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 创建数据加载器
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 4. 定义模型
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 实例化模型并将其发送到设备
model = NeuralNet(input_size, hidden_sizes, output_size).to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 5. 训练模型
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 将图片和标签移动到设备
        images = images.reshape(-1, input_size).to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item()}')

# 测试模型，在测试阶段关闭梯度
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

model.eval()

# 准备一个示例输入，这将用于模型的跟踪
example = torch.rand(1, input_size).to(device)

# 使用 'torch.jit.trace' 方法追踪模型，或者使用 'torch.jit.script' 方法脚本化模型
# 跟踪模型
traced_script_module = torch.jit.trace(model, example)

# 保存追踪后的模型到磁盘
traced_script_module.save(f"minist_{num_epochs}.pt")

Epoch [1/50], Step [100/938], Loss: 0.5177665948867798
Epoch [1/50], Step [200/938], Loss: 0.517952024936676
Epoch [1/50], Step [300/938], Loss: 0.33616718649864197
Epoch [1/50], Step [400/938], Loss: 0.2392079383134842
Epoch [1/50], Step [500/938], Loss: 0.15293897688388824
Epoch [1/50], Step [600/938], Loss: 0.37737101316452026
Epoch [1/50], Step [700/938], Loss: 0.3027612268924713
Epoch [1/50], Step [800/938], Loss: 0.14483892917633057
Epoch [1/50], Step [900/938], Loss: 0.2886595129966736
Epoch [2/50], Step [100/938], Loss: 0.20182228088378906
Epoch [2/50], Step [200/938], Loss: 0.07629455626010895
Epoch [2/50], Step [300/938], Loss: 0.36338475346565247
Epoch [2/50], Step [400/938], Loss: 0.1835208237171173
Epoch [2/50], Step [500/938], Loss: 0.2499009370803833
Epoch [2/50], Step [600/938], Loss: 0.1669710874557495
Epoch [2/50], Step [700/938], Loss: 0.3042776584625244
Epoch [2/50], Step [800/938], Loss: 0.22158721089363098
Epoch [2/50], Step [900/938], Loss: 0.06121711805462837
Ep

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 卷积层
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(128)
        # Dropout层
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        # 全连接层
        self.fc1 = nn.Linear(128 * 7 * 7, 256) # 更正全连接层输入特征数
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = F.relu(self.batchnorm2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.batchnorm3(self.conv3(x)))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


# 训练模型
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'\rTrain Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}',end='')

# 测试模型
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss()(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * accuracy:.0f}%)\n')
    return test_loss, correct, accuracy  # 添加返回值

# 定义 JIT 模型保存函数
def save_jit_model(model, epoch, device, example, filename_prefix):
    model.eval()
    traced_script_module = torch.jit.trace(model, example)
    traced_script_module.save(f"{filename_prefix}_{epoch}.pt")
    print(f"JIT compiled model saved at epoch {epoch}")

# 设置种子和设备
torch.manual_seed(1)
device = torch.device("mps")

# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 加载数据集
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transform),
    batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transform),
    batch_size=1000, shuffle=True)

# 实例化模型和优化器
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

num_epochs = 200
save_interval = 100
best_loss = float('inf')
best_epoch = 0

# 准备示例输入
example = torch.rand(1, 1, 28, 28).to(device)

# 训练和测试
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test_loss, correct, accuracy = test(model, device, test_loader)

    # 每隔save_interval个epoch保存一次JIT编译后的模型
    if epoch % save_interval == 0 and epoch != num_epochs:
        save_jit_model(model, epoch, device, example, "minist_cnn_jit_checkpoint")

    # 保存表现最好的JIT编译后的模型
    if test_loss < best_loss:
        best_loss = test_loss
        best_epoch = epoch
        save_jit_model(model, 0, device ,example, "best")

# 最后保存经过训练的JIT编译后的模型
save_jit_model(model, num_epochs, device, example, "minist_cnn_jit")

print(f"Training complete! Best JIT compiled model saved with epoch {best_epoch} and loss {best_loss:.4f}.")

Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.066318
Test set: Average loss: 0.0001, Accuracy: 9828/10000 (98%)

JIT compiled model saved at epoch 0
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.036448
Test set: Average loss: 0.0000, Accuracy: 9866/10000 (99%)

JIT compiled model saved at epoch 0
Train Epoch: 3 [57600/60000 (96%)]	Loss: 0.024496
Test set: Average loss: 0.0000, Accuracy: 9898/10000 (99%)

JIT compiled model saved at epoch 0
Train Epoch: 4 [57600/60000 (96%)]	Loss: 0.029727
Test set: Average loss: 0.0000, Accuracy: 9919/10000 (99%)

JIT compiled model saved at epoch 0
Train Epoch: 5 [57600/60000 (96%)]	Loss: 0.007744
Test set: Average loss: 0.0000, Accuracy: 9911/10000 (99%)

Train Epoch: 6 [57600/60000 (96%)]	Loss: 0.078606
Test set: Average loss: 0.0000, Accuracy: 9918/10000 (99%)

JIT compiled model saved at epoch 0
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.025351
Test set: Average loss: 0.0000, Accuracy: 9920/10000 (99%)

JIT compiled model saved at epoch 0
Train Epoch: 8